In [1]:
import geopandas as gpd
import pandas as pd
import boto3

In [2]:
bucket_name = 'opera-validation-datasets'
root_folder_name = 'calval-database-dev/'

In [3]:
image_name = '20211011_155455_52_2262'

In [4]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')

In [5]:
collocation = gpd.read_file('Collocation_HLS_Planet_2021.geojson')

In [6]:
collocation.head()

,hls_Date,hls_ID,hls_Cloud_Cover,hls_Footprint,hls_AOI_intersect,pl_Date,pl_ID,pl_Cloud_Cover,pl_AOI_intersect,instrument,provider,resolution,hls_pl_AOI_intersect,timeDelta_days,chip_ID,chip_rID,geometry
0,2021-09-02T22:13:28,HLS.L30.T04WDB.2021245T221328.v2.0,80.0,"{'type': 'Polygon', 'coordinates': [[[-161.436...",100.0,2021-09-01T22:52:36,20210901_225236_74_105e,100.0,0.040170,PS2.SD,planetscope,3.0,0.040170,0.972816,3_38,99.0,"POLYGON ((-159.24215 69.43651, -159.41164 69.2..."
1,2021-09-02T22:13:28,HLS.L30.T04WDB.2021245T221328.v2.0,80.0,"{'type': 'Polygon', 'coordinates': [[[-161.436...",100.0,2021-09-02T21:28:04,20210902_212804_69_2460,100.0,5.340357,PSB.SD,planetscope,3.0,5.340357,0.031520,3_38,99.0,"POLYGON ((-159.58347 69.15572, -159.79242 68.9..."
2,2021-09-02T22:13:28,HLS.L30.T04WDB.2021245T221328.v2.0,80.0,"{'type': 'Polygon', 'coordinates': [[[-161.436...",100.0,2021-09-02T21:28:02,20210902_212802_21_2460,99.0,24.620320,PSB.SD,planetscope,3.0,24.620320,0.031549,3_38,99.0,"POLYGON ((-159.41800 69.30108, -159.62926 69.1..."
3,2021-09-02T22:13:28,HLS.L30.T04WDB.2021245T221328.v2.0,80.0,"{'type': 'Polygon', 'coordinates': [[[-161.436...",100.0,2021-09-02T21:27:59,20210902_212759_73_2460,96.0,2.336268,PSB.SD,planetscope,3.0,2.336268,0.031577,3_38,99.0,"POLYGON ((-159.25102 69.44763, -159.46527 69.2..."
4,2021-09-02T22:13:28,HLS.L30.T04WDB.2021245T221328.v2.0,80.0,"{'type': 'Polygon', 'coordinates': [[[-161.436...",100.0,2021-09-02T21:32:24,20210902_213224_71_2442,100.0,3.175964,PSB.SD,planetscope,3.0,3.175964,0.028510,3_38,99.0,"POLYGON ((-160.06368 69.17413, -160.27703 68.9..."


In [7]:
search = collocation[(collocation.pl_ID == image_name)&(collocation.hls_pl_AOI_intersect  >= 50)]
search.head()

,hls_Date,hls_ID,hls_Cloud_Cover,hls_Footprint,hls_AOI_intersect,pl_Date,pl_ID,pl_Cloud_Cover,pl_AOI_intersect,instrument,provider,resolution,hls_pl_AOI_intersect,timeDelta_days,chip_ID,chip_rID,geometry
39186,2021-10-11T16:39:05,HLS.S30.T15QYU.2021284T162221.v2.0,2.0,"{'type': 'Polygon', 'coordinates': [[[-90.1033...",100.0,2021-10-11T15:54:55,20211011_155455_52_2262,0.0,70.640659,PSB.SD,planetscope,3.0,70.640659,0.030674,3_10,91.0,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."


In [8]:
image = search

In [9]:
image = image.drop(columns=['hls_Date','hls_Cloud_Cover','hls_Footprint','hls_AOI_intersect','hls_pl_AOI_intersect','chip_rID'])
image.head()

,hls_ID,pl_Date,pl_ID,pl_Cloud_Cover,pl_AOI_intersect,instrument,provider,resolution,timeDelta_days,chip_ID,geometry
39186,HLS.S30.T15QYU.2021284T162221.v2.0,2021-10-11T15:54:55,20211011_155455_52_2262,0.0,70.640659,PSB.SD,planetscope,3.0,0.030674,3_10,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."


In [10]:
image = image.rename(columns = {'hls_ID':'collocated_dswx','pl_Date':'datetime','pl_ID':'image_name','pl_Cloud_Cover':'cloud_cover','pl_AOI_intersect':'site_coverage','chip_ID':'site_name'})


In [11]:
image.head()

,collocated_dswx,datetime,image_name,cloud_cover,site_coverage,instrument,provider,resolution,timeDelta_days,site_name,geometry
39186,HLS.S30.T15QYU.2021284T162221.v2.0,2021-10-11T15:54:55,20211011_155455_52_2262,0.0,70.640659,PSB.SD,planetscope,3.0,0.030674,3_10,"POLYGON ((-90.50851 16.35841, -90.54697 16.176..."


In [12]:
#image.to_file('image.geojson',driver='GeoJSON')

In [13]:
#response = s3.upload_file('image.geojson',bucket_name,root_folder_name+'image.geojson')

In [14]:
image_bytes = bytes(image.to_json(drop_id=True).encode('UTF-8'))
s3object = s3.Object(bucket_name,root_folder_name+'image.geojson')
s3object.put(Body=image_bytes)

{'ResponseMetadata': {'RequestId': '35MNPTZZ2KDF4ABA',
  'HostId': 'uyaUJfGnYZC97gC4MAlQZdiDkjX5abFep8R8r6Sq+1SbjKnLK1Y3I7z89pI/gGhgJ8n9t43Jz8U=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'uyaUJfGnYZC97gC4MAlQZdiDkjX5abFep8R8r6Sq+1SbjKnLK1Y3I7z89pI/gGhgJ8n9t43Jz8U=',
   'x-amz-request-id': '35MNPTZZ2KDF4ABA',
   'date': 'Tue, 21 Jun 2022 16:53:21 GMT',
   'etag': '"2ea1b5b577635adba680e769e5b4597f"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2ea1b5b577635adba680e769e5b4597f"'}